<a href="https://colab.research.google.com/github/putopavel/practical-application-sentiment-analysis-deep-learning/blob/master/beto_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install --quiet transformers nlp
    from google.colab import drive
    import os
    # This will prompt for authorization.
    drive.mount('/content/drive')
    drive_path = "/content/drive/My Drive/TFM"
    os.chdir(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import nlp
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [3]:
def get_model(model_name):
    config = AutoConfig.from_pretrained(model_name, num_labels=3)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [4]:
label_to_int = {
    "AGAINST": 0,
    "FAVOR": 1,
    "NEUTRAL": 2
}
int_to_label = {v: k for k, v in label_to_int.items()}

def tokenize(batch):
    return tokenizer(batch["tweet"], max_length = 256, truncation = True, padding = "max_length")


def get_dataset(data_files):
    dataset = nlp.load_dataset(
        "csv",
        delimiter="\t",
        data_files=data_files,
    )
    dataset.remove_column_("id_str")
    dataset.rename_column_("TWEET", "tweet")
    dataset.rename_column_("LABEL", "label")
    dataset = dataset.map(lambda example: {"label": label_to_int[example["label"]]})
    dataset = dataset.map(tokenize)
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
    return dataset

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


def get_trainer(model, dataset):
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=10,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        evaluate_during_training=True,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"]
    )
    return trainer

In [6]:
def print_classification_report(trainer, dataset):
    prediction = trainer.predict(dataset["test"])
    transform_to_label = np.vectorize(lambda i: int_to_label[i])
    test_labels = transform_to_label(prediction.label_ids)
    test_preds = transform_to_label(prediction.predictions.argmax(-1))
    print(classification_report(test_labels, test_preds))

# Spanish dataset

In [7]:
model_name = "dccuchile/bert-base-spanish-wwm-cased"
model, tokenizer = get_model(model_name)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

In [8]:
%%timeit -r 1 -n 1
spanish_data_files = {
    "train": "spanish_train.csv",
    "validation": "spanish_val.csv",
    "test": "spanish_test.csv"
}

spanish_dataset = get_dataset(spanish_data_files)
spanish_trainer = get_trainer(model, spanish_dataset)
spanish_trainer.train()
print_classification_report(spanish_trainer, spanish_dataset)

Using custom data configuration default


/usr/local/lib/python3.6/dist-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Trainer is attempting to log a value of "[0.74050294 0.68960953 0.75692308]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.68311945 0.71467764 0.9010989 ]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.80841121 0.66624041 0.65251989]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.75696767 0.70457516 0.79320113]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.7238806  0.72058824 0.85106383]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.7932243  0.68925831 0.74270557]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.72782875 0.72981178 0.78609626]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.76379974 0.69479769 0.79245283]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.69509346 0.7685422  0.77984085]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.



              precision    recall  f1-score   support

     AGAINST       0.72      0.77      0.74       829
       FAVOR       0.75      0.71      0.73       807
     NEUTRAL       0.86      0.81      0.83       380

    accuracy                           0.75      2016
   macro avg       0.78      0.76      0.77      2016
weighted avg       0.76      0.75      0.75      2016

1 loop, best of 1: 33min 56s per loop


# Catalan dataset

In [9]:
model, tokenizer = get_model(model_name)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

In [10]:
%%timeit -r 1 -n 1
catalan_data_files = {
    "train": "catalan_train.csv",
    "validation": "catalan_val.csv",
    "test": "catalan_test.csv"
}

catalan_dataset = get_dataset(catalan_data_files)
catalan_trainer = get_trainer(model, catalan_dataset)
catalan_trainer.train()
print_classification_report(catalan_trainer, catalan_dataset)

Using custom data configuration default


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bbe0cc4ee012d2fb/0.0.0/d27f9d4163bc98ad11a8c6b35120f1486e488f0fba0736cae84fcc51c291c35e. Subsequent calls will reuse this data.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Trainer is attempting to log a value of "[0.74254413 0.75095785 0.68803946]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.74119077 0.73225405 0.7265625 ]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.74390244 0.7706422  0.65339578]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.75452842 0.74728087 0.69392523]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.77336748 0.73       0.69230769]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.73658537 0.76539974 0.69555035]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.76547416 0.74191328 0.67990074]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.7162593  0.78115942 0.72295515]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.82195122 0.70642202 0.64168618]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.



              precision    recall  f1-score   support

     AGAINST       0.68      0.70      0.69       752
       FAVOR       0.73      0.76      0.75       804
     NEUTRAL       0.72      0.63      0.67       454

    accuracy                           0.71      2010
   macro avg       0.71      0.70      0.70      2010
weighted avg       0.71      0.71      0.71      2010

1 loop, best of 1: 33min 46s per loop


# Combined dataset

In [14]:
model, tokenizer = get_model(model_name)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

In [15]:
%%timeit -r 1 -n 1
combined_data_files = {
    "train": "combined_train.csv",
    "validation": "combined_val.csv",
    "test": "combined_test.csv"
}

combined_dataset = get_dataset(combined_data_files)
combined_trainer = get_trainer(model, combined_dataset)
combined_trainer.train()
print_classification_report(combined_trainer, combined_dataset)

Using custom data configuration default


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-555ace17379ed833/0.0.0/d27f9d4163bc98ad11a8c6b35120f1486e488f0fba0736cae84fcc51c291c35e. Subsequent calls will reuse this data.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Trainer is attempting to log a value of "[0.68211307 0.7132544  0.69925926]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.70723895 0.6438999  0.86446886]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.65871122 0.79935275 0.58706468]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.73609935 0.7102742  0.72972973]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.69844671 0.72537112 0.79881657]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.77804296 0.69579288 0.67164179]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

Trainer is attempting to log a value of "[0.74322733 0.74412617 0.74337427]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.73372093 0.74007682 0.77388964]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.75298329 0.74822006 0.71517413]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.73024523 0.73563218 0.73976405]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.74124155 0.69051675 0.83673469]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.71957041 0.78705502 0.66293532]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.74711674 0.73438527 0.73640743]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.70675891 0.74615898 0.82434515]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.79236277 0.72297735 0.66542289]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[0.74155995 0.72827497 0.73856209]" of type <class 'numpy.ndarray'> for key "eval_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.72386364 0.72969461 0.77823691]" of type <class 'numpy.ndarray'> for key "eval_precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.7601432  0.72686084 0.70273632]" of type <class 'numpy.ndarray'> for key "eval_recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.



              precision    recall  f1-score   support

     AGAINST       0.72      0.72      0.72      1581
       FAVOR       0.72      0.76      0.74      1611
     NEUTRAL       0.78      0.73      0.76       834

    accuracy                           0.74      4026
   macro avg       0.74      0.73      0.74      4026
weighted avg       0.74      0.74      0.74      4026

1 loop, best of 1: 1h 6min 36s per loop
